This is a setup/test/demonstration notebook for the `AutoExposure` effect in Scopesim. This effect splits the requested total exposure time into NDIT subexposures of integration time DIT such that the maximum counts in a single subexposure does not exceed a certain fill fraction of the detector full well. The final readout is the sum over the NDIT subexposures, i.e. corresponds to the total requested exposure time.

The notebook uses the `irdb/METIS` configuration. The observed source is blank sky, except for the last example where a star of 0 mag is used (Vega).

In [ ]:
import scopesim as sim
sim.bug_report()

# Edit this path if you have a custom install directory, otherwise comment it out.
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../../../"

If you haven't got the instrument packages yet, uncomment the following cell. 

In [ ]:
# sim.download_packages(["METIS", "ELT", "Armazones"])

# Imaging LM-band

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=["img_lm"])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
src = sim.source.source_templates.empty_sky()

In [ ]:
metis.observe(src)

For `AutoExposure` to work the exposure time has to be given explicitely as a parameter to the `readout` method. If this is not done, the default values for `DIT` and `NDIT` will be used. The following is for an exposure time of 1 second. The resulting readout is divided by `NDIT` to produce the average over the `NDIT` subexposures; this allows direct comparison to the detector full well. 

In [ ]:
outimg = metis.readout(exptime=1)[0][1].data 
outimg = outimg / metis.cmds[metis['summed_exposure'].meta['ndit']]

full_well = metis.cmds["!DET.full_well"]
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

The same with a much larger exposure time of 1000 seconds:

In [ ]:
outimg = metis.readout(exptime = 1000)[0][1].data 
outimg = outimg / metis.cmds[metis['summed_exposure'].meta['ndit']]

full_well = metis.cmds["!DET.full_well"]
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

The desired fill fraction can be changed with the argument `fill_frac`. The default value of 75 per cent is a typical good value that keeps the detector counts within the linear regime.

In [ ]:
outimg = metis.readout(exptime = 1000, fill_frac=0.9)[0][1].data 
outimg = outimg / metis.cmds[metis['summed_exposure'].meta['ndit']]

full_well = metis.cmds["!DET.full_well"]
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

# Imaging N-band

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['img_n'])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout(exptime=1)[0][1].data 
outimg = outimg / metis.cmds[metis['summed_exposure'].meta['ndit']]

full_well = metis.cmds["!DET.full_well"]
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:9.1f}".format(outimg.max()))
print("Detector full well:                 {:9.1f}".format(full_well))
print("Fill fraction:                      {:9.1f} per cent".format(100 * outimg.max()/ full_well))

# Long-slit spectroscopy

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['lss_l'])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout(exptime=3600.)[0][1].data 
outimg = outimg / metis.cmds[metis['summed_exposure'].meta['ndit']]

full_well = metis.cmds["!DET.full_well"]

print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

# What happens when the source saturates the detector?
We take an N-band image of Vega. `DIT` is automatically set to the minimum value supported by the detector, but the centre of the star still saturates the detector. In the final image, the star's profile is capped at the full well of the detector.

In [ ]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=["img_n"])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
src = sim.source.source_templates.star()

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout(exptime=1)[0][1].data
outimg = outimg / metis.cmds[metis['summed_exposure'].meta['ndit']]

full_well = metis.cmds["!DET.full_well"]

print("\nResult\n======")
print("Maximum value in readout (per DIT): {:9.1f}".format(outimg.max()))
print("Detector full well:                 {:9.1f}".format(full_well))
print("Fill fraction:                      {:9.1f} per cent".format(100 * outimg.max()/ full_well))

Plot a cut through the star to show how its peak saturates the detector.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(outimg[950:1100, 1024])

In [ ]:
npix = (outimg >= full_well).sum()
print("Number of saturated pixels:", npix)

The default values for the detector full well in the various modes reflects our current best knowledge of the properties of the actual METIS detectors. These values can be changed as in the following example, but be aware that this makes the simulations unrealistic.

In [ ]:
full_well = 1000 * metis.cmds["!DET.full_well"]
outimg = metis.readout(exptime=1, full_well=full_well)[0][1].data
outimg = outimg / metis.cmds[metis['summed_exposure'].meta['ndit']]

print("\nResult\n======")
print("Maximum value in readout (per DIT): {:9.1f}".format(outimg.max()))
print("Detector full well:                 {:9.1f}".format(full_well))
print("Fill fraction:                      {:9.1f} per cent".format(100 * outimg.max()/ full_well))